In [1]:
import pandas as pd

In [2]:
count_1 = pd.read_csv('./label_result/count_1gram.csv')
count_2 = pd.read_csv('./label_result/count_2gram.csv')
count_3 = pd.read_csv('./label_result/count_3gram.csv')
count_4 = pd.read_csv('./label_result/count_4gram.csv')
count_5 = pd.read_csv('./label_result/count_5gram.csv')

In [3]:
df = pd.concat([count_1, count_2, count_3, count_4, count_5], ignore_index=True)

# 결과 확인
print(df)

                                        ngram  P_count  N_count
0                                    대통령실/NNG      707      137
1                                     대변인/NNG      322      161
2                                      정부/NNG    16931    14319
3                                   제2금융권/NNG      527      625
4                                     고금리/NNG     4659     3517
...                                       ...      ...      ...
87944  당일/NNG 채권시장/NNG 미국/NNG 채권시장/NNG 강세/NNG       39        9
87945     금일/NNG 전략/NNG 국고채/NNG 금리/NNG 목표/NNG       71       10
87946       부담/NNG 인해/VV 약세/NNG 진행/NNG 전망/NNG       14        9
87947   전략/NNG 국고채/NNG 금리/NNG 목표/NNG 매도전략/NNG       41        8
87948   국고채/NNG 금리/NNG 목표/NNG 매도전략/NNG 제안/NNG       41        8

[87949 rows x 3 columns]


In [4]:
import pandas as pd
import numpy as np

df.set_index('ngram', inplace=True)

# 열 추가
k = 0.5
df['P(word|P)'] = (df['P_count'] + k )/ (df['P_count'].sum() + 2 * k)
df['P(word|N)'] = (df['N_count'] + k )/ (df['N_count'].sum() + 2 * k)
df['log(P(word|P))'] = np.log(df['P(word|P)'])
df['log(P(word|N))'] = np.log(df['P(word|N)'])

# 폴라리티 스코어
words = [index_value for index_value in df.index]

# log 활용 underflow 방지
final_P = np.log(df['P_count'].sum() / (df['P_count'].sum() + df['N_count'].sum()) )
final_N = np.log(df['N_count'].sum() / (df['P_count'].sum() + df['N_count'].sum()) )
scores = []
for token in words:
    final_P += df['log(P(word|P))'][token]
    final_N += df['log(P(word|N))'][token]
    final_P = np.exp(final_P)
    final_N = np.exp(final_N)
    a = final_P / (final_P + final_N)
    b = final_N / (final_P + final_N)
    scores.append(a / b)
    # final_P = np.log(df['P_count'].sum() / (df['P_count'].sum() + df['N_count'].sum()) )
    # final_N = np.log(df['N_count'].sum() / (df['P_count'].sum() + df['N_count'].sum()) )
df['Polarity'] = scores
df
# 시드 단어 분류
P_seed_words = []
N_seed_words = []
for word, pola in zip(df.index, df['Polarity']):
    if pola > 1.3 :
        P_seed_words.append(word)
    elif pola < 0.7 :
        N_seed_words.append(word)
    else :
        pass

In [5]:
P_seed_words

['대통령실/NNG',
 '대변인/NNG',
 '차질/NNG',
 '대통령/NNG',
 '주재/NNG',
 '민생/NNG',
 '대환/NNG',
 '영업이익/NNG',
 '뛰어넘/VV',
 '고물가/NNG',
 '인상기/NNG',
 '세대/NNG',
 '인상/NNG',
 '금융부채/NNG',
 '오를수록/VV',
 '분위/NNG',
 '노트/NNG',
 '고령자/NNG',
 '달할/VV',
 '보입니다/VV',
 '주담대/NNG',
 '스트레스/NNG',
 '차주/NNG',
 '내년/NNG',
 '최종/NNG',
 '원리금균등상환방식/NNG',
 '택할/VV',
 '씨/NNG',
 '올리/VV',
 '올릴/VV',
 '액티브/NNG',
 '총괄/NNG',
 '챗/NNG',
 '주도주/NNG',
 '공급망/NNG',
 '푸드/NNG',
 '모디/NNG',
 '총리/NNG',
 '공장/NNG',
 '침체/NNG',
 '압력/NNG',
 '침체국면/NNG',
 '제안/NNG',
 '희소성/NNG',
 '채굴/NNG',
 '커질/VA',
 '식품/NNG',
 '스토어/NNG',
 '로봇/NNG',
 '자율주행/NNG',
 '연대/NNG',
 '오전/NNG',
 '공익/NNG',
 '방카슈랑스/NNG',
 '소비자보호법/NNG',
 '분담/NNG',
 '맞춰질/VV',
 '페드워치/NNG',
 '연이/VV',
 '벌어지/VV',
 '역전폭/NNG',
 '하단/NNG',
 '짚/VV',
 '상방/NNG',
 '사용량/NNG',
 '인플레이션/NNG',
 '부연/NNG',
 '신선식품/NNG',
 '물가/NNG',
 '연말/NNG',
 '긴축/NNG',
 '물가연동채권/NNG',
 '정상화/NNG',
 '오버/NNG',
 '전무/NNG',
 '자산시장/NNG',
 'PCE/NNG',
 '발걸음/NNG',
 '나을/VA',
 '대체투자/NNG',
 '사업장/NNG',
 '블라인드/NNG',
 '병/NNG',
 '평가손실/NNG',
 '임대형/NNG',
 '코어/NNG',
 

In [6]:
N_seed_words

['투자부문/NNG',
 '이득/NNG',
 'ELS/NNG',
 '주가연계증권/NNG',
 '추천/NNG',
 '스트레스금리/NNG',
 '하한/NNG',
 '가산금리/NNG',
 '여윳돈/NNG',
 '물/VV',
 '펀드/NNG',
 '자산배분/NNG',
 '수혜/NNG',
 '투자성향/NNG',
 '센트/NNG',
 '집중투자/NNG',
 '주식형/NNG',
 '국유기업/NNG',
 '기대수익률/NNG',
 '결정짓/VV',
 '디플레이션/NNG',
 'ROE/NNG',
 '굵직/NNG',
 '판매사/NNG',
 '인간/NNG',
 '갤럭시/NNG',
 '감사원/NNG',
 '판매자/NNG',
 '파생상품/NNG',
 '지켜보/VV',
 '파생/NNG',
 '입찰/NNG',
 '대차/NNG',
 '인하/NNG',
 '공매도/NNG',
 '내수/NNG',
 '절상/NNG',
 '완만/NNG',
 '탈피/NNG',
 '선진/NNG',
 '갭/NNG',
 '엔저/NNG',
 '제로금리정책/NNG',
 '고시금리/NNG',
 '양도성예금증서/NNG',
 'CD/NNG',
 '증권사/NNG',
 '딜러/NNG',
 '재간접펀드/NNG',
 '이탈리아/NNG',
 '제로금리/NNG',
 '양적완화/NNG',
 '내릴/VV',
 '금리동결/NNG',
 '농산물/NNG',
 '차별화/NNG',
 '안전자산/NNG',
 '공고히/MAG',
 '철저/NNG',
 '리그/NNG',
 '심정/NNG',
 '강변/NNG',
 '유로/NNG',
 '충당부채/NNG',
 '촉매제/NNG',
 '디커플링/NNG',
 '지점/NNG',
 '낙찰/NNG',
 '연방준비제도이사회/NNG',
 '신용등급/NNG',
 '성향/NNG',
 '문구/NNG',
 '내려도/VV',
 '무방/NNG',
 '표명/NNG',
 '삽입/NNG',
 '급속히/MAG',
 '울퉁불퉁/MAG',
 '견해/NNG',
 '공사비/NNG',
 '연면적/NNG',
 '달라질/VA',
 '완/VV',
 '뱅/MAG',

In [8]:
df_P = pd.read_csv('./label_result/df_P_1gram_test.csv')
df_N = pd.read_csv('./label_result/df_N_1gram_test.csv')

In [9]:
def dict_date(date_list, token_list) :
    date_token = {}
    for date, tokens in zip(date_list, token_list):
        if date not in date_token:
            date_token[date] = [tokens]
        else:
            date_token[date].append(tokens)
    return date_token
# 사용법 csv파일들 tolist() 사용하셔서 list로 만들고 date_token자리에 넣으시면 됩니다.
N_date_list = df_N['date'].tolist()
N_token_list = df_N['tokens'].tolist()
N_date_token = dict_date(N_date_list, N_token_list)

In [17]:
date_sents_list = {}
for date, sents in N_date_token.items():
    for sent in sents:
        # date_sents_list에 해당 날짜의 리스트가 없으면 새로 생성하고 문장을 추가
        if date not in date_sents_list:
            date_sents_list[date] = [sent]
        # 이미 해당 날짜의 리스트가 존재하면 해당 리스트에 문장을 추가
        else:
            date_sents_list[date].append(sent)

# 문장의 톤 계산
date_sents_tone = {}
for date, sents_list in date_sents_list.items():
    pos = 0
    neg = 0
    for sent in sents_list:
        for word in sent:
            if word in P_seed_words:
                pos += 1
            elif word in N_seed_words:
                neg += 1
    # 날짜별 톤 계산 후 저장
    if pos != 0:
        date_sents_tone[date] = neg / pos
    else:
        date_sents_tone[date] = 0.0

In [18]:
date_sents_tone 

{'2024-03-21': 0.0}

In [16]:
date_sents_list = {}
for date, sents in N_date_token.items():
    for sent in sents:
        
        if date not in date_sents_list:
            date_sents_list[date] = sent
        else:
            date_sents_list[date].append(sent)
# 문장의 톤 계산
date_sents_tone = {}
for date, sents_list in date_sents_list.items():
    print(sents_list)
    pos = 0
    neg = 0
    for sent in sents_list:
        for word in sent:
            if word in P_seed_words:
                pos += 1
            elif word in N_seed_words:
                neg += 1
    print(pos)
    print(neg)
    # 날짜별 톤 계산 후 저장
    if pos != 0:
        date_sents_tone[date] = neg / pos
    else:
        date_sents_tone[date] = 0.0

AttributeError: 'str' object has no attribute 'append'

In [13]:
date_sents_list

{'2024-03-21': ["[('지난/VV',), ('일/NNG',), ('오후/NNG',), ('업/NNG',), ('비트/NNG',), ('라운지/NNG',), ('비트코인/NNG',), ('시세/NNG',), ('나오/VV',), ('있/VX',)]",
  "[('뉴스/NNG',), ('최근/NNG',), ('하락세/NNG',), ('보이/VV',), ('가상화폐/NNG',), ('비트코인/NNG',), ('일/NNG',), ('현지/NNG',), ('시간/NNG',), ('미/NNG',), ('연방준비제도/NNG',), ('Fed/NNG',), ('연준/NNG',), ('연내/NNG',), ('차례/NNG',), ('기준금리/NNG',), ('인하/NNG',), ('전망/NNG',), ('유지/NNG',), ('이상/NNG',), ('급등/NNG',)]",
  "[('미/NNG',), ('가상화폐/NNG',), ('거래소/NNG',), ('코인/NNG',), ('베이스/NNG',), ('따르/VV',), ('미/NNG',), ('시간/NNG',), ('이날/NNG',), ('오후/NNG',), ('서부/NNG',), ('시간/NNG',), ('오후/NNG',), ('현재/MAG',), ('비트코인/NNG',), ('가격/NNG',), ('전/NNG',), ('급등/NNG',), ('달러/NNG',), ('거래/NNG',)]",
  "[('앞서/MAG',), ('비트코인/NNG',), ('지난/VV',), ('일/NNG',), ('역대/NNG',), ('최고가/NNG',), ('달러/NNG',), ('기록/NNG',), ('뒤/NNG',), ('일/NNG',), ('간/VV',), ('하락/NNG',), ('곡선/NNG',), ('그렸/VV',)]",
  "[('전날/NNG',), ('고점/NNG',), ('대비/NNG',), ('하락/NNG',), ('달러/NNG',), ('선/NNG',), ('위협/NNG',)]",
  "[('지난/VV',), (

In [28]:
date_sents_tone

{'2024-03-21': 0.0}

In [ ]:
import ast

# 문자열을 파이썬 리스트로 변환
N_token_list = ast.literal_eval(N_token_list)

# 결과 출력
print(N_token_list[0])

In [ ]:
# tokens = NBC 문장 ngram들
import numpy as np

def NBC_test(df, labels, tokens):
    # 전체 라벨에 대해서 확률들 계산
    # 가장 큰 확률의 라벨 반환
    probs = {}
    idxs = df.index # ngram들
    for label in labels:
        probs[label] = np.log(df[label].sum() / df[labels].sum().sum())
        for token in tokens:
            if token not in idxs: continue
            probs[label] += df[f'log_{label}'][token]
        probs[label] = np.exp(probs[label])

    results = []
    for label in labels:
        results.append( probs[label] / sum(probs.values()) )

    return labels[ np.argmax(results) ]

In [ ]:
# 문장 = ['ngram1', 'ngram2', ...]
# 긍정_사전 = ['ngram1', 'ngram2']
# 부정_사전 = ['ngram4', 'ngram5']

pos = 0
neg = 0
for ngram in N_date_token:
    if ngram in P_seed_words:
        pos += 1
    elif ngram in N_seed_words:
        neg += 1

문장tone = '긍정' if pos > neg else '부정'

문서tone = ['문장1tone', '문장2tone', '문장3tone'] # 긍정 2, 부정 1
문서tone = 2/3 # 긍정쪽으로 0 ~ 1사이의 값 -> 기준금리 범위를 0~1사이로 조정하여 비교